In [1]:
import unittest
from typing import List, Tuple

def simple_overlap_rerank(query: str, documents: List[Tuple[str, str]]) -> List[Tuple[str, str, int]]:
    """
    Rerank documents by simple token overlap count with the query.
    Returns list of (doc_id, text, score) sorted by score desc then original order (stable).
    """
    q_tokens = query.lower().split()
    q_set = set(q_tokens)
    scored = []
    for doc_id, text in documents:
        tokens = text.lower().split()
        score = sum(1 for t in tokens if t in q_set)
        scored.append((doc_id, text, score))
    # stable sort by negative score
    scored.sort(key=lambda x: -x[2])
    return scored

class TestTextReranker(unittest.TestCase):
    def setUp(self):
        self.query = "cat on mat"
        self.documents = [
            ("d1", "The cat sits on the mat"),
            ("d2", "Quantum physics and relativity"),
            ("d3", "Cat and mat story about a playful cat on a mat"),
        ]

    def test_rerank_order(self):
        reranked = simple_overlap_rerank(self.query, self.documents)
        ordered_ids = [d[0] for d in reranked]
        self.assertEqual(ordered_ids, ["d3", "d1", "d2"])
        # Check scores strictly descending
        scores = [d[2] for d in reranked]
        self.assertTrue(all(scores[i] >= scores[i+1] for i in range(len(scores)-1)))

    def test_zero_overlap(self):
        q = "neural network"
        reranked = simple_overlap_rerank(q, self.documents)
        # All scores should be 0
        self.assertTrue(all(s == 0 for *_, s in reranked))

    def test_stability(self):
        # Add another doc with identical score to d1
        docs = self.documents + [("d4", "Cat on mat")]
        reranked = simple_overlap_rerank(self.query, docs)
        # d3 still first
        self.assertEqual(reranked[0][0], "d3")
        # d1 should appear before d4 because of stability (both same score, d1 came first)
        ids = [d[0] for d in reranked]
        self.assertTrue(ids.index("d1") < ids.index("d4"))

if __name__ == "__main__":
    unittest.main(argv=["-v"], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
